In [1]:
import pandas as pd
from lib import standardize_item_numbers
import numpy as np

In [2]:
def clean():
    dfa = pd.read_csv("../data/police_reports/electronic_police_report_2018_2022.csv")
    dfa = dfa.pipe(standardize_item_numbers, ["item_number"])

    dfb = pd.read_csv("../data/real_time_crime_center/rtcc.csv")
    dfb["rtcc_requested"] = "1"
    dfb = dfb.pipe(standardize_item_numbers, ["item_number"])

    df = pd.merge(dfa, dfb, on="item_number")
    return dfa

In [3]:
df = clean()

C:\Users\PC\AppData\Local\Temp\ipykernel_16072\1424771964.py:2: DtypeWarning: Columns (14,20,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  dfa = pd.read_csv("../data/police_reports/electronic_police_report_2018_2022.csv")


In [4]:
df.columns

Index(['Unnamed: 0', 'item_number', 'district', 'location', 'disposition',
       'signal_type', 'signal_description', 'occurred_date_time',
       'charge_code', 'charge_description', 'offender_race', 'offender_gender',
       'offender_age', 'offender_number', 'person_type', 'victim_race',
       'victim_gender', 'victim_age', 'victim_number', 'victim_fatal_status',
       'hate_crime', 'report_type', 'year', 'offenderid', 'offenderstatus',
       'persontype'],
      dtype='object')

In [5]:
df.loc[:, "location"] = df.location.str.replace(r"(\w+)$", r"\1, New Orleans, Louisiana")

requested = df[["item_number", "location"]]

requested_1 = requested[:5000]
requested_2 = requested[5000:10000]
requested_3 = requested[10000:15000]
requested_4 = requested[15000:]

# requested_1.to_csv("../data/real_time_crime_center/rtcc_locations_1.csv", index=False)
# requested_2.to_csv("../data/real_time_crime_center/rtcc_locations_2.csv", index=False)
# requested_3.to_csv("../data/real_time_crime_center/rtcc_locations_3.csv", index=False)
# requested_4.to_csv("../data/real_time_crime_center/rtcc_locations_4.csv", index=False)

C:\Users\PC\AppData\Local\Temp\ipykernel_16072\1998154791.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, "location"] = df.location.str.replace(r"(\w+)$", r"\1, New Orleans, Louisiana")


In [6]:
requested_1_coordinates = pd.read_csv("../data/real_time_crime_center/rtcc_locations_1_coordinates.csv")
requested_1_df = pd.read_csv("../data/real_time_crime_center/rtcc_locations_1.csv")
rtcc_1 = pd.concat([requested_1_df, requested_1_coordinates], axis=1)

In [7]:
requested_2_coordinates_df = pd.read_csv("../data/real_time_crime_center/rtcc_locations_2_coordinates.csv")
requested_2_df = pd.read_csv("../data/real_time_crime_center/rtcc_locations_2.csv")
rtcc_2 = pd.concat([requested_2_df, requested_2_coordinates_df], axis=1)

In [8]:
requested_3_coordinates_df = pd.read_csv("../data/real_time_crime_center/rtcc_locations_3_coordinates.csv")
requested_3_df = pd.read_csv("../data/real_time_crime_center/rtcc_locations_3.csv")
rtcc_3 = pd.concat([requested_3_df, requested_3_coordinates_df], axis=1)

In [9]:
requested_4_coordinates_df = pd.read_csv("../data/real_time_crime_center/rtcc_locations_4_coordinates.csv")
requested_4_df = pd.read_csv("../data/real_time_crime_center/rtcc_locations_4.csv")
rtcc_4 = pd.concat([requested_4_df, requested_4_coordinates_df], axis=1)

In [10]:
coordinates_df = pd.concat([rtcc_1, rtcc_2, rtcc_3, rtcc_4], axis=0)

In [11]:
# coordinates_df.to_csv("../data/real_time_crime_center/rtcc_coordinates.csv", index=False)

In [12]:
coordinates_df = pd.read_csv("../data/real_time_crime_center/rtcc_coordinates.csv")
coordinates_df = coordinates_df.rename(columns={"0": "latitude", "1": "longitude"})
coordinates_df = coordinates_df[["item_number", "latitude", "longitude"]]

In [13]:
df = pd.merge(df, coordinates_df, on="item_number")

In [14]:
df = df.drop_duplicates()

In [15]:
df['occurred_date_time'] = sorted(pd.to_datetime(df['occurred_date_time']))
df['hour'] = df['occurred_date_time'].dt.hour
df = df[~((df.hour.fillna("") == ""))]
df.loc[:, "hour"] = df.hour.astype(str)

In [16]:
df.disposition.unique()

array(['CLOSED', 'OPEN', 'UNFOUNDED'], dtype=object)

In [17]:
dates = df[["occurred_date_time", "latitude", "longitude", "hour", "item_number", "disposition"]]
dates = dates[~((dates.occurred_date_time.fillna("") == ""))]

def split_dates(df):
    data = df.occurred_date_time.astype(str).str.extract(r"^(\w+-\w+-\w+) (.+)")
    df.loc[:, "occurred_date"] = data[0]
    df.loc[:, "occurred_time"] = data[1]    
    return df

dates = dates.pipe(split_dates)
dates.to_csv("../data/real_time_crime_center/dates_6.csv", index=False)
dates

# night = dates[dates.hour.isin(["19", "20", "21", "22", "23", "00", "1", "2", "3", "4", "5", "6"])]
# night["night"]  = "night"

# day = dates[~((dates.hour.isin(["19", "20", "21", "22", "23", "00", "1", "2", "3", "4", "5", "6"])))]
# day["day"] = 1


# night.to_csv("../data/real_time_crime_center/night.csv", index=False)
# day.to_csv("../data/real_time_crime_center/day.csv", index=False)

,occurred_date_time,latitude,longitude,hour,item_number,disposition,occurred_date,occurred_time
0,2018-01-08 16:30:00,29.948867,-90.089342,16,d2077118,CLOSED,2018-01-08,16:30:00
2,2018-01-08 16:30:00,29.948867,-90.089342,16,d2077118,CLOSED,2018-01-08,16:30:00
4,2018-01-26 02:00:00,29.965823,-90.036381,2,e1274818,OPEN,2018-01-26,02:00:00
5,2018-01-26 02:00:00,30.072179,-89.943564,2,l2194418,OPEN,2018-01-26,02:00:00
9,2018-01-26 02:00:00,30.072179,-89.943564,2,l2194418,OPEN,2018-01-26,02:00:00
...,...,...,...,...,...,...,...,...
102906,2022-11-18 02:10:00,29.954712,-90.069155,2,b2385022,OPEN,2022-11-18,02:10:00
102907,2022-11-18 02:10:00,29.964152,-90.057798,2,a0325822,CLOSED,2022-11-18,02:10:00
102908,2022-11-18 02:10:00,30.076163,-89.858976,2,j2225922,OPEN,2022-11-18,02:10:00
102909,2022-11-18 19:15:00,29.973199,-90.039967,19,a1349122,OPEN,2022-11-18,19:15:00
